In [1]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
try:
  # Disable all GPUS
  tf.config.set_visible_devices(physical_devices[1], 'GPU')
  visible_devices = tf.config.get_visible_devices()
  print('Visible devices:', visible_devices)
except:
  print('Invalid device or cannot modify virtual devices once initialized.')
  pass

Visible devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [2]:
#!pip3 install optuna

In [3]:
# Import all the necessary libraries.
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
import numpy.random as rnd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import BayesianRidge
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
import csv
from autogluon.tabular import TabularDataset, TabularPredictor
import autokeras as ak
from autokeras import StructuredDataClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import load_model
from datetime import datetime
import tensorflow as tf
from sklearn.metrics import accuracy_score
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM, Input
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers import Conv1D
from keras.layers import MaxPooling1D, AveragePooling1D
from keras.backend import clear_session
import optuna
from sklearn.decomposition import PCA
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

## Getting the data and splitting the sets

In [4]:
number_of_ports = 5
threshold = 10

In [5]:
# Reset pseudo random generator to a known value so that results are reproducible.
seed = 42
np.random.seed(seed)

In [6]:
dados = pd.read_csv('./classical_channel_samples_SINR.csv', header=None)
dados.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.059398,0.183232,1.080309,2.614261,4.584051,6.749800,8.862194,10.691750,12.054431,12.830632,...,23.051142,21.468345,19.913783,18.554710,17.555975,17.061368,17.174608,17.942390,19.342121,21.276428
1,23.778188,30.222646,36.359023,41.707733,45.852210,48.479536,49.409203,48.607128,46.183609,42.376109,...,9.341135,7.158377,5.033784,3.273157,2.148745,1.864735,2.530243,4.143316,6.588110,9.645830
2,7.832415,7.090963,6.494046,6.230958,6.451343,7.240751,8.604879,10.464771,12.663762,14.985351,...,7.779974,4.201972,1.725694,0.367445,0.015321,0.452600,1.393285,2.525027,3.553824,4.244767
3,2.709786,3.666249,4.566536,5.313171,5.822715,6.036252,5.926975,5.503941,4.811494,3.924439,...,6.793335,5.646034,4.686189,4.033345,3.794001,4.044375,4.815511,6.082973,7.762920,9.715667
4,10.603570,7.808421,6.219126,6.101364,7.555183,10.500586,14.683635,19.702510,25.050711,30.172516,...,13.615531,16.162296,18.415918,20.167117,21.241454,21.520567,20.958055,19.587812,17.523448,14.948487


In [7]:
print(dados.size)
print(dados.shape)
dados.describe()

100000000
(1000000, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
count,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1.000000e+06,1000000.000000,1000000.000000,1000000.000000,...,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1.000000e+06,1.000000e+06,1000000.000000,1000000.000000
mean,10.003086,10.004461,10.005724,10.006757,10.007477,10.007848,1.000789e+01,10.007660,10.007274,10.006859,...,10.013202,10.014060,10.014483,10.014552,10.014383,10.014111,1.001386e+01,1.001371e+01,10.013720,10.013862
std,10.003076,10.000057,9.998053,9.997400,9.998204,10.000222,1.000285e+01,10.005251,10.006590,10.006328,...,10.007767,10.010679,10.014165,10.017877,10.021456,10.024579,1.002701e+01,1.002862e+01,10.029415,10.029448
min,0.000006,0.000033,0.000021,0.000023,0.000006,0.000004,4.811054e-07,0.000002,0.000002,0.000004,...,0.000009,0.000014,0.000007,0.000004,0.000005,0.000004,9.557120e-07,9.344143e-07,0.000004,0.000002
25%,2.882745,2.882236,2.887322,2.886715,2.878279,2.879955,2.881660e+00,2.882988,2.882563,2.880368,...,2.882248,2.883979,2.880040,2.885831,2.880396,2.880352,2.877562e+00,2.878891e+00,2.882343,2.885722
50%,6.934941,6.933017,6.939023,6.939193,6.940237,6.943029,6.938943e+00,6.935458,6.936254,6.932065,...,6.941200,6.946095,6.945471,6.947915,6.939139,6.944599,6.940263e+00,6.944825e+00,6.934469,6.939888
75%,13.843788,13.865951,13.875038,13.882409,13.886576,13.884771,1.388077e+01,13.880488,13.868526,13.881989,...,13.885061,13.875917,13.871919,13.878928,13.889511,13.880936,1.387604e+01,1.387670e+01,13.871021,13.870399
max,149.928490,158.053010,160.410605,156.827589,147.622207,133.570093,1.294800e+02,131.301800,140.857127,156.519784,...,148.344790,156.795340,163.599276,164.152879,165.848467,162.582232,1.547072e+02,1.504642e+02,140.904372,137.098610


In [8]:
def reduzir_array(arr, novo_tamanho):
    indices = np.linspace(0, arr.shape[1]-1, novo_tamanho, dtype=int)
    array_reduzido = arr[indices]
    return array_reduzido, indices

dados_, idx = reduzir_array(dados, number_of_ports)
dados_.describe()

,0,24,49,74,99
count,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000
mean,10.003086,10.011016,9.999887,10.018376,10.013862
std,10.003076,10.022521,9.987231,10.013229,10.029448
min,0.000006,0.000019,0.000011,0.000008,0.000002
25%,2.882745,2.879628,2.875682,2.880966,2.885722
50%,6.934941,6.944056,6.932545,6.933767,6.939888
75%,13.843788,13.869075,13.874761,13.886951,13.870399
max,149.928490,159.658812,135.762065,175.306850,137.098610


In [9]:
X = dados_.iloc[:, :].to_numpy()
Xindices = idx
y = dados.iloc[:, :].to_numpy()                                                 

In [10]:
del dados_
del idx
del dados

In [11]:
# Split the whole set into random training and validation set.
X_train_, X_val_, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=seed)

print('Total:', len(y))
print('Train', len(y_train))
print('Val', len(y_val))

Total: 1000000
Train 800000
Val 200000


In [12]:
X.shape

(1000000, 5)

In [13]:
Xindices.shape

(5,)

In [14]:
del X
del y

## Saving project

In [36]:
version = '15102024b'
project_name = 'cnn_lstm_fama_sinr_regression_'+str(number_of_ports)+'_ports_'+version
print(project_name)

cnn_lstm_fama_sinr_regression_5_ports_15102024b


In [28]:
# Referência 
# https://github.com/ashrefm/multi-label-soft-f1/blob/master/Multi-Label%20Image%20Classification%20in%20TensorFlow%202.0.ipynb

@tf.function
def macro_soft_f1(y, y_hat):
    """Compute the macro soft F1-score as a cost (average 1 - soft-F1 across all labels).
    Use probability values instead of binary predictions.
    
    Args:
        y (int32 Tensor): targets array of shape (BATCH_SIZE, N_LABELS)
        y_hat (float32 Tensor): probability matrix from forward propagation of shape (BATCH_SIZE, N_LABELS)
        
    Returns:
        cost (scalar Tensor): value of the cost function for the batch
    """
    y = tf.cast(y, tf.float32)
    y_hat = tf.cast(y_hat, tf.float32)
    tp = tf.reduce_sum(y_hat * y, axis=0)
    fp = tf.reduce_sum(y_hat * (1 - y), axis=0)
    fn = tf.reduce_sum((1 - y_hat) * y, axis=0)
    soft_f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    cost = 1 - soft_f1 # reduce 1 - soft-f1 in order to increase soft-f1
    macro_cost = tf.reduce_mean(cost) # average on all labels
    return macro_cost

In [38]:
def getOp(Xindices, y_val_pred, y_val):
    '''Estimate outage probability for regression models'''
    
    # Após a predição do modelo, verificar se a escolha foi melhor que a referência (i.e., melhor que as N portas observadas)
    # Caso contrário, se uma porta observada tiver sido melhor que a predição, escolhe a porta observada.
    
    # Monta um array com o valor da porta observada
    observed_values = np.zeros((y_val.shape[0], 100)) - np.float64('inf')
    observed_values[:, Xindices] = y_val[:, Xindices]

    pred_values = np.argmax(y_val_pred, axis=1)

    predicted_values = np.zeros((y_val.shape[0], 100)) - np.float64('inf')
    predicted_values[np.arange(len(pred_values)), pred_values] = y_val[np.arange(len(pred_values)), pred_values]
    
    # Pega o máximo entre as portas observadas e preditadas
    maximo_entre_arrays = np.maximum(observed_values, predicted_values)

    print("Max Arrays:", maximo_entre_arrays.shape)
    
    # Indice (Porta) da melhor predição/observada
    best_pred_obs_port = np.argmax(maximo_entre_arrays, axis=1)

    print("Best Pred Obs:", best_pred_obs_port.shape)

    print("Best_Ports_Len = ", len(best_pred_obs_port))
    
    # Valor do dado real na porta escolhida pela predição
    values_for_predicted_classes = maximo_entre_arrays[np.arange(len(y_val)), best_pred_obs_port]

    print("Values Preds:", values_for_predicted_classes.shape)
    
    # True or False para os valores que ficaram acima do limiar do gamma_th
    # acima_do_limiar = values_for_predicted_classes > gamma_th_linear
    # acima_do_limiar = values_for_predicted_classes > gamma_th
    acima_do_limiar = values_for_predicted_classes > threshold

    print("Above thresh:", acima_do_limiar.shape)
    
    # Probabilidade de outage
    p_out = 1.0 - (np.sum(acima_do_limiar) / len(y_val))
    
    return p_out

In [39]:
# choose a number of time steps
n_steps = 1
n_features = number_of_ports
n_seq = 1

In [40]:
def objective(trial):

    global X_train_
    global X_val_
    
    # Clear clutter from previous Keras session graphs.
    clear_session()    

    activation = trial.suggest_categorical('activation', ['relu', 'linear', 'sigmoid', 'tanh'])
    cnn = trial.suggest_categorical('cnn', [0, 1, 2])
    filters = trial.suggest_int('filters', 1, 512)
    kernel_size = trial.suggest_categorical('kernel_size', [1, 2, 3, 4, 5])
    learning_rate = trial.suggest_float('lr', 1e-5, 1e-1, log=True)
    dense = trial.suggest_categorical('dense', [0, 1, 2, 3])
    lstm = trial.suggest_categorical('lstm', [1, 2, 3])
    
    pool = trial.suggest_categorical('pool', ['max', 'avg'])
    if(pool == 'max'):
        pool_layer = MaxPooling1D(pool_size=1)
    else:
        pool_layer = AveragePooling1D(pool_size=1)

    optimizer = trial.suggest_categorical('optimizer', ['adam', 'sgd', 'nadam'])
    if optimizer=='adam':
        opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer=='nadam':
        opt = tf.keras.optimizers.Nadam(learning_rate=learning_rate)
    elif optimizer=='sgd':
        opt = tf.keras.optimizers.SGD(learning_rate=learning_rate)

    scaler = trial.suggest_categorical('scaler', ['std', 'minmax', 'none'])
    if(scaler == 'std'):
        scl = StandardScaler()
    else:
        scl = MinMaxScaler(feature_range=(-1, 1))
    
    if(scaler=='std' or scaler=='minmax'):
        X_train__ = scl.fit_transform(X_train_)
        X_val__ = scl.transform(X_val_)
    else:
        X_train__ = X_train_
        X_val__ = X_val_
    
    pca = trial.suggest_categorical('pca', ['yes', 'no'])
    if(pca == 'yes'):
        pCA = PCA()
        X_train___ = pCA.fit_transform(X_train__)
        X_val___ = pCA.transform(X_val__)
    else:
        X_train___ = X_train__
        X_val___ = X_val__
    
    
    del X_train__
    del X_val__

    if(cnn>0):
        X_train = X_train___.reshape((X_train___.shape[0], n_seq, n_steps, n_features))
        X_val = X_val___.reshape((X_val___.shape[0], n_seq, n_steps, n_features))
    else:
        X_train = X_train___.reshape((X_train___.shape[0], n_steps, n_features))
        X_val = X_val___.reshape((X_val___.shape[0], n_steps, n_features))

    del X_train___
    del X_val___
    
    # define model
    model = Sequential()
    
    if(cnn>0):
        sp=(None, n_steps, n_features)
    else:
        sp=(n_steps, n_features)
    model.add(Input(shape=sp))
    
    if(cnn>0):

        for i in range(cnn):
            model.add(TimeDistributed(Conv1D(filters=filters, kernel_size=kernel_size, padding='same', activation='relu')))
            model.add(TimeDistributed(pool_layer))
        model.add(TimeDistributed(Flatten()))
    
    for i in range(lstm):
        cells = trial.suggest_int('cells_'+str(i), 1, 100)
        if(lstm > 1 and i < lstm-1):
            return_sequences=True
        else:
            return_sequences=False
        model.add(LSTM(cells, activation=activation, return_sequences=return_sequences))
    for i in range(dense):
        nodes = trial.suggest_int('nodes_'+str(i), 1, 300)
        model.add(Dense(nodes, activation='relu'))
        dropout = trial.suggest_categorical('dropout_'+str(i), [0.0, 0.1, 0.2, 0.3, 0.4, 0.5])
        model.add(Dropout(dropout))
    model.add(Dense(100, activation='linear'))

    loss = tf.keras.losses.MeanSquaredError()
    
    model.compile(loss=loss,
                  optimizer=opt,
                  metrics=['mae'])

    es_cb = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)    

    model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=2048, epochs=50, verbose=True, callbacks=[es_cb])
    
    # Evaluate the model accuracy on the validation set.
    #score = model.evaluate(X_val, y_val, verbose=0)
    y_val_pred = model.predict(X_val, batch_size=2048)
    
    op = getOp(Xindices, y_val_pred, y_val)
    
    return op

## Running the studies

In [ ]:
# Run optimization
study_name = "study_"+project_name
storage_name = 'sqlite:///{}.db'.format(study_name)

study = optuna.create_study(direction='minimize', study_name=study_name, storage=storage_name, load_if_exists=True) # reducing the loss and the complexity
print(f"Sampler is {study.sampler.__class__.__name__}")
study.optimize(objective, n_trials=200)

print(study.best_params)

[I 2024-10-15 14:13:19,917] Using an existing study with name 'study_cnn_lstm_fama_sinr_regression_5_ports_15102024b' instead of creating a new one.


Sampler is TPESampler
Epoch 1/50
391/391 [==============================] - 7s 12ms/step - loss: 115.1329 - mae: 7.4537 - val_loss: 73.9450 - val_mae: 6.0373
Epoch 2/50
391/391 [==============================] - 4s 11ms/step - loss: 78.1978 - mae: 6.2249 - val_loss: 66.4206 - val_mae: 5.5684
Epoch 3/50
391/391 [==============================] - 4s 11ms/step - loss: 73.5534 - mae: 6.0001 - val_loss: 64.3372 - val_mae: 5.4660
Epoch 4/50
391/391 [==============================] - 4s 11ms/step - loss: 71.6655 - mae: 5.9049 - val_loss: 63.6275 - val_mae: 5.3934
Epoch 5/50
391/391 [==============================] - 4s 11ms/step - loss: 70.7560 - mae: 5.8547 - val_loss: 63.1096 - val_mae: 5.3916
Epoch 6/50
391/391 [==============================] - 4s 10ms/step - loss: 70.2337 - mae: 5.8241 - val_loss: 63.0272 - val_mae: 5.3616
Epoch 7/50
391/391 [==============================] - 4s 11ms/step - loss: 69.8659 - mae: 5.8015 - val_loss: 62.9562 - val_mae: 5.3562
Epoch 8/50
391/391 [============

[I 2024-10-15 14:16:51,399] Trial 1 finished with value: 0.08098000000000005 and parameters: {'activation': 'linear', 'cnn': 0, 'filters': 237, 'kernel_size': 2, 'lr': 0.0002605947799523198, 'dense': 2, 'lstm': 2, 'pool': 'max', 'optimizer': 'adam', 'scaler': 'none', 'pca': 'no', 'cells_0': 46, 'cells_1': 61, 'nodes_0': 188, 'dropout_0': 0.5, 'nodes_1': 30, 'dropout_1': 0.2}. Best is trial 1 with value: 0.08098000000000005.


Max Arrays: (200000, 100)
Best Pred Obs: (200000,)
Best_Ports_Len =  200000
Values Preds: (200000,)
Above thresh: (200000,)
Epoch 1/50
391/391 [==============================] - 10s 17ms/step - loss: 209.4414 - mae: 8.4994 - val_loss: 121.2580 - val_mae: 7.0604
Epoch 2/50
391/391 [==============================] - 6s 16ms/step - loss: 108.9190 - mae: 6.9977 - val_loss: 101.2610 - val_mae: 7.0750
Epoch 3/50
391/391 [==============================] - 6s 16ms/step - loss: 100.8555 - mae: 7.2215 - val_loss: 99.6350 - val_mae: 7.2980
Epoch 4/50
391/391 [==============================] - 6s 16ms/step - loss: 100.3533 - mae: 7.3455 - val_loss: 99.5956 - val_mae: 7.3478
Epoch 5/50
391/391 [==============================] - 6s 16ms/step - loss: 100.3420 - mae: 7.3661 - val_loss: 99.5959 - val_mae: 7.3526
Epoch 6/50
391/391 [==============================] - 6s 16ms/step - loss: 100.3421 - mae: 7.3685 - val_loss: 99.5960 - val_mae: 7.3514
Epoch 7/50
391/391 [==============================] - 6s 

[I 2024-10-15 14:22:11,422] Trial 2 finished with value: 0.11848999999999998 and parameters: {'activation': 'linear', 'cnn': 2, 'filters': 239, 'kernel_size': 5, 'lr': 0.022958779280784113, 'dense': 1, 'lstm': 3, 'pool': 'avg', 'optimizer': 'adam', 'scaler': 'std', 'pca': 'yes', 'cells_0': 62, 'cells_1': 43, 'cells_2': 80, 'nodes_0': 166, 'dropout_0': 0.1}. Best is trial 1 with value: 0.08098000000000005.


...

Max Arrays: (200000, 100)
Best Pred Obs: (200000,)
Best_Ports_Len =  200000
Values Preds: (200000,)
Above thresh: (200000,)
Epoch 1/50
391/391 [==============================] - 7s 13ms/step - loss: 200.6108 - mae: 10.0134 - val_loss: 199.3378 - val_mae: 9.9871
Epoch 2/50
391/391 [==============================] - 5s 12ms/step - loss: 200.5773 - mae: 10.0118 - val_loss: 199.3028 - val_mae: 9.9854
Epoch 3/50
391/391 [==============================] - 5s 12ms/step - loss: 200.5423 - mae: 10.0100 - val_loss: 199.2672 - val_mae: 9.9836
Epoch 4/50
391/391 [==============================] - 5s 12ms/step - loss: 200.5070 - mae: 10.0083 - val_loss: 199.2312 - val_mae: 9.9818
Epoch 5/50
391/391 [==============================] - 5s 12ms/step - loss: 200.4715 - mae: 10.0065 - val_loss: 199.1953 - val_mae: 9.9800
Epoch 6/50
391/391 [==============================] - 5s 12ms/step - loss: 200.4358 - mae: 10.0047 - val_loss: 199.1593 - val_mae: 9.9782
Epoch 7/50
391/391 [============================

[I 2024-10-15 14:38:37,707] Trial 7 finished with value: 0.08381499999999997 and parameters: {'activation': 'relu', 'cnn': 1, 'filters': 235, 'kernel_size': 3, 'lr': 1.3040911610025205e-05, 'dense': 3, 'lstm': 2, 'pool': 'avg', 'optimizer': 'sgd', 'scaler': 'std', 'pca': 'yes', 'cells_0': 48, 'cells_1': 29, 'nodes_0': 252, 'dropout_0': 0.2, 'nodes_1': 146, 'dropout_1': 0.4, 'nodes_2': 191, 'dropout_2': 0.1}. Best is trial 5 with value: 0.07785500000000001.


Max Arrays: (200000, 100)
Best Pred Obs: (200000,)
Best_Ports_Len =  200000
Values Preds: (200000,)
Above thresh: (200000,)
Epoch 1/50
391/391 [==============================] - 5s 7ms/step - loss: 200.3465 - mae: 10.0012 - val_loss: 198.8785 - val_mae: 9.9651
Epoch 2/50
391/391 [==============================] - 2s 5ms/step - loss: 199.8912 - mae: 9.9787 - val_loss: 198.3984 - val_mae: 9.9414
Epoch 3/50
391/391 [==============================] - 2s 5ms/step - loss: 199.3865 - mae: 9.9539 - val_loss: 197.8629 - val_mae: 9.9149
Epoch 4/50
391/391 [==============================] - 2s 5ms/step - loss: 198.8159 - mae: 9.9258 - val_loss: 197.2479 - val_mae: 9.8846
Epoch 5/50
391/391 [==============================] - 2s 5ms/step - loss: 198.1484 - mae: 9.8931 - val_loss: 196.5132 - val_mae: 9.8485
Epoch 6/50
391/391 [==============================] - 2s 5ms/step - loss: 197.3335 - mae: 9.8534 - val_loss: 195.5971 - val_mae: 9.8037
Epoch 7/50
391/391 [==============================] - 2s 5m

[I 2024-10-15 14:40:25,176] Trial 8 finished with value: 0.11848999999999998 and parameters: {'activation': 'tanh', 'cnn': 0, 'filters': 259, 'kernel_size': 1, 'lr': 0.00018792949714907796, 'dense': 2, 'lstm': 1, 'pool': 'max', 'optimizer': 'sgd', 'scaler': 'std', 'pca': 'yes', 'cells_0': 3, 'nodes_0': 10, 'dropout_0': 0.5, 'nodes_1': 172, 'dropout_1': 0.5}. Best is trial 5 with value: 0.07785500000000001.


Max Arrays: (200000, 100)
Best Pred Obs: (200000,)
Best_Ports_Len =  200000
Values Preds: (200000,)
Above thresh: (200000,)
Epoch 1/50
391/391 [==============================] - 5s 8ms/step - loss: 200.5127 - mae: 10.0085 - val_loss: 199.0990 - val_mae: 9.9750
Epoch 2/50
333/391 [========================>.....] - ETA: 0s - loss: 200.0055 - mae: 9.9856

In [ ]:
print("Best params: ", study.best_params)
print("Best value: ", study.best_value)
print("Best Trial: ", study.best_trial)

## Training the models

In [20]:
params_5 = {'activation': 'sigmoid', 'cnn': 0, 'filters': 31, 'kernel_size': 4, 'lr': 0.0007075516779091285, 'dense': 0, 'lstm': 1, 'pool': 'avg', 'optimizer': 'sgd', 'scaler': 'minmax', 'pca': 'no', 'cells_0': 84}

params_teste = {'activation': 'sigmoid', 'cnn': 0, 'filters': 486, 'kernel_size': 4, 'lr': 0.03717307353131541, 'dense': 3, 'lstm': 3, 'pool': 'avg', 'optimizer': 'nadam', 'scaler': 'none', 'pca': 'yes', 'cells_0': 15, 'cells_1': 33, 'cells_2': 35, 'nodes_0': 120, 'dropout_0': 0.5, 'nodes_1': 76, 'dropout_1': 0.5, 'nodes_2': 112, 'dropout_2': 0.5}

In [21]:
def getModel(params, X_train_, X_val_, epochs=50, batch_size=2048):
    
    # Clear clutter from previous Keras session graphs.
    clear_session()    

    activation = params['activation']
    cnn = params['cnn']
    filters = params['filters']
    kernel_size = params['kernel_size']
    learning_rate = params['lr']
    dense = params['dense']
    lstm = params['lstm']
    
    pool = params['pool']
    if(pool == 'max'):
        pool_layer = MaxPooling1D(pool_size=1)
    else:
        pool_layer = AveragePooling1D(pool_size=1)

    optimizer = params['optimizer']
    if optimizer=='adam':
        opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer=='nadam':
        opt = tf.keras.optimizers.Nadam(learning_rate=learning_rate)
    elif optimizer=='sgd':
        opt = tf.keras.optimizers.SGD(learning_rate=learning_rate)

    scaler = params['scaler']
    if(scaler == 'std'):
        scl = StandardScaler()
    else:
        scl = MinMaxScaler(feature_range=(-1, 1))

    if(scaler=='std' or scaler=='minmax'):
        X_train__ = scl.fit_transform(X_train_)
        X_val__ = scl.transform(X_val_)
    else:
        X_train__ = X_train_
        X_val__ = X_val_

    pca = params['pca']
    if(pca == 'yes'):
        pCA = PCA()
        X_train___ = pCA.fit_transform(X_train__)
        X_val___ = pCA.transform(X_val__)
    else:
        X_train___ = X_train__
        X_val___ = X_val__

    del X_train__
    del X_val__

    if(cnn>0):
        X_train = X_train___.reshape((X_train___.shape[0], n_seq, n_steps, n_features))
        X_val = X_val___.reshape((X_val___.shape[0], n_seq, n_steps, n_features))
    else:
        X_train = X_train___.reshape((X_train___.shape[0], n_steps, n_features))
        X_val = X_val___.reshape((X_val___.shape[0], n_steps, n_features))

    del X_train___
    del X_val___
    
    # define model
    model = Sequential()
    if(cnn>0):
        sp=(None, n_steps, n_features)
    else:
        sp=(n_steps, n_features)
    model.add(Input(shape=sp))
    
    if(cnn>0):
        model.add(TimeDistributed(Conv1D(filters=filters, kernel_size=kernel_size, padding='same', activation='relu')))
        model.add(TimeDistributed(pool_layer))
        model.add(TimeDistributed(Flatten()))
    
    for i in range(lstm):
        cells = params['cells_'+str(i)]
        if(lstm > 1 and i < lstm-1):
            return_sequences=True
        else:
            return_sequences=False
        model.add(LSTM(cells, activation=activation, return_sequences=return_sequences))
    for i in range(dense):
        nodes = params['nodes_'+str(i)]
        model.add(Dense(nodes, activation='relu'))
        dropout = params['dropout_'+str(i)]
        model.add(Dropout(dropout))
    model.add(Dense(100, activation='linear'))
    
    loss = tf.keras.losses.MeanSquaredError()
    
    model.compile(loss=loss,
                  optimizer=opt,
                  metrics=['mae'])
    
    es_cb = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)    
    
    model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=2048, epochs=40, verbose=True, callbacks=[es_cb])
    
    y_val_pred = model.predict(X_val, batch_size=2048)
    
    op = getOp(Xindices, y_val_pred, y_val)

    return model, op, y_val_pred

In [24]:
model, op, y_val_pred = getModel(params_teste, X_train_, X_val_)

Epoch 1/40
391/391 [==============================] - 11s 20ms/step - loss: 101.9299 - mae: 7.2363 - val_loss: 89.9587 - val_mae: 6.9026
Epoch 2/40
391/391 [==============================] - 7s 19ms/step - loss: 92.7634 - mae: 7.0151 - val_loss: 90.4387 - val_mae: 6.8435
Epoch 3/40
391/391 [==============================] - 7s 19ms/step - loss: 91.4830 - mae: 6.9607 - val_loss: 88.3596 - val_mae: 6.7886
Epoch 4/40
391/391 [==============================] - 7s 19ms/step - loss: 89.2013 - mae: 6.8550 - val_loss: 85.4807 - val_mae: 6.6853
Epoch 5/40
391/391 [==============================] - 7s 19ms/step - loss: 87.5356 - mae: 6.7936 - val_loss: 85.0293 - val_mae: 6.6452
Epoch 6/40
391/391 [==============================] - 8s 19ms/step - loss: 85.4561 - mae: 6.7049 - val_loss: 82.1191 - val_mae: 6.5302
Epoch 7/40
391/391 [==============================] - 7s 19ms/step - loss: 83.3919 - mae: 6.6162 - val_loss: 81.2195 - val_mae: 6.4644
Epoch 8/40
391/391 [==============================] -

In [25]:
print(op)

0.11848999999999998


In [ ]:
len(y_val_pred)

In [ ]:
print(y_val_pred.shape)
y_val_pred.view()

In [ ]:
dataframe = pd.DataFrame.from_records(y_val_pred)

In [ ]:
dataframe.describe()

In [ ]:
dataframe.head()

In [ ]:
model.summary()

In [ ]:
model_name = "model_"+project_name

print(model_name)

In [ ]:
if(1):
    try:
        model.save(model_name, save_format="tf")
    except Exception:
        model.save(model_name+".h5")

In [ ]:
loaded_model = load_model(model_name, custom_objects={'macro_soft_f1':macro_soft_f1})

In [ ]:
y_train_pred = loaded_model.predict(X_train)
    
y_val_pred = loaded_model.predict(X_val)

In [ ]:
y_train_pred_bin = np.where(y_train_pred < 0.5, 0, 1)
y_val_pred_bin = np.where(y_val_pred < 0.5, 0, 1)

In [ ]:
acc_val = 0
for line in range(y_val.shape[0]):
    for column in range(y_val.shape[1]):

        if(y_val_pred_bin[line, column] == y_val[line, column]):
            acc_val += 1
acc_val /= (5*y_val.shape[0])


acc_train = 0
for line in range(y_train.shape[0]):
    for column in range(y_train.shape[1]):

        if(y_train_pred_bin[line, column] == y_train[line, column]):
            acc_train += 1
acc_train /= (5*y_train.shape[0])

print('Acc train:', acc_train)
print('Acc val:', acc_val)

In [ ]:
score_train=loaded_model.evaluate(X_train, y_train)
score_val=loaded_model.evaluate(X_val, y_val)
print(score_train)
print(score_val)

In [ ]:
top_5_predicted_classes = np.argsort(y_val_pred, axis=1)[:, -5:]  # Índices das top 5 classes previstas

In [ ]:
top_5_predicted_classes

In [ ]:
# Verifique se a classe predita está entre as top 5 mais prováveis
is_top_5 = np.any(top_5_predicted_classes == np.argsort(y_val, axis=1)[:, -5:], axis=1)

# Calcule a acurácia top-5
top_5_accuracy = np.sum(is_top_5) / len(y_val)

print(f'A acurácia top-5 foi: {top_5_accuracy * 100}%')

In [ ]:
dados = pd.read_csv('./classical_channel_samples.csv', header=None)
dados.head()

In [ ]:
dados_teste = dados.iloc[:, :].to_numpy()

train_df, dados_teste = train_test_split(dados_teste, test_size=0.2, random_state=42)

In [ ]:
predicted_classes = []
best_pred_obs_ports = []

# for n in range(len(loaded_model)):
for n in range(1):

    #Criando um dataset para cada valor do N de entrada
    x_data = []
    observed_ports = []

    
    arr, indices = reduzir_array(dados_teste, number_of_ports)
    x_data.append(arr)


    x_data = np.array(x_data)

    
    predictions_dB = y_val_pred

    # Após a predição do modelo, verificar se a escolha foi melhor que a referência (Max N_init portas observadas)
    # Caso contrário, uma porta observada, tiver sido melhor que a predição, escolhe a porta observada

    # Monta um array com o valor da porta observada
    observed_values = np.zeros((len(dados_teste),100)) - 100
    observed_values[:,indices] = dados_teste[:,indices]


    # Monta um array com o valor real da porta preditada
    pred_classes = np.argmax(predictions_dB, axis=1)

    predicted_values = np.zeros((len(dados_teste),100)) - 100
    predicted_values[np.arange(len(pred_classes)), pred_classes] = dados_teste[np.arange(len(pred_classes)), pred_classes]

    # Pega o máximo entre as portas observadas e preditadas
    maximo_entre_arrays = np.maximum(observed_values, predicted_values)

    # Indice (Porta) da melhor predição/observada
    best_pred_obs_port = np.argmax(maximo_entre_arrays, axis=1)


    predicted_classes.append(np.argmax(predictions_dB, axis=1))
    best_pred_obs_ports.append(best_pred_obs_port)


In [ ]:
#OP LSTM2
predicted_lstm_op = []

# for i in range(len(1)):
# Valor do dado real na porta escolhida pela predição
values_for_predicted_classes = dados_teste[np.arange(len(dados_teste)), best_pred_obs_ports]

# True or False para os valores que ficaram acima do limiar do gamma_th
# acima_do_limiar = values_for_predicted_classes > gamma_th_linear
# acima_do_limiar = values_for_predicted_classes > gamma_th
acima_do_limiar = values_for_predicted_classes > 5

# Probabilidade de Outage
p_out = 1 - np.sum(acima_do_limiar) / len(dados_teste)
print(f'A probabilidade de Outage (OP) foi {"{:.5e}".format(p_out)} = {"{:.2e}".format(p_out).replace("e", "*10^")}\n')

predicted_lstm_op.append(p_out)

In [ ]:
op=getOp(y_val, y_val_pred, number_of_ports)
print(op)

# LSTM OP

5  Portas - 0.23139500000000002

6  Portas - 0.20364000000000004

7  Portas - 0.18842000000000003

8  Portas - 0.17806

9  Portas - 0.17127499999999996

10 Portas - 0.16713

11 Portas - 0.16464999999999996

13 Portas - 0.16351000000000004

15 Portas - 0.16324499999999997

20 Portas - 0.16295000000000004

25 Portas - 0.16242999999999996

25 Portas - 0.16235